In [3]:
import os 
from dotenv import load_dotenv
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

from langchain_groq import ChatGroq

In [4]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.runnables import RunnableConfig
from langchain_core.messages import HumanMessage,AIMessage

In [5]:
store={}
def get_session(session_id:str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

In [6]:
model = ChatGroq(api_key=groq_api_key,model_name="llama-3.1-8b-instant")
chat_with_history=RunnableWithMessageHistory(model,get_session,)

In [7]:
config = {"configurable": {"session_id": "chat1"}}

response = chat_with_history.invoke([HumanMessage(content="Hey hi, my name is Kakashi hatake and i am a gen ai intern")],config=config)
print(response.content)

Kakashi Hatake, the legendary Jonin from the Hidden Leaf Village. I'm glad to meet you, Gen AI intern. As a knowledgeable being, I'm curious, what kind of projects have you been working on during your internship?


In [8]:
response=chat_with_history.invoke([HumanMessage(content="Do you know my name and what i am currently doing")],config=config)
print(response.content)

I'm familiar with the name Kakashi Hatake, the former Team 7 leader and a renowned Jonin from the Hidden Leaf Village, known for his exceptional skills with the Sharingan eye and his iconic ninja headband. However, I'm not aware of the specific context or details about your current occupation as a "Gen AI intern."


In [9]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

chain_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant, answer these question "),
    MessagesPlaceholder(variable_name="messages")
])


In [12]:
chain=chain_prompt|model
chain.invoke({
    "messages":[HumanMessage(content="Hi my name is harshit hatake")]
})

AIMessage(content='Nice to meet you, Harshit Hatake. How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 53, 'total_tokens': 72, 'completion_time': 0.023141744, 'completion_tokens_details': None, 'prompt_time': 0.002641314, 'prompt_tokens_details': None, 'queue_time': 0.054878735, 'total_time': 0.025783058}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--1486532f-e33f-4f0f-b75b-703862563740-0', usage_metadata={'input_tokens': 53, 'output_tokens': 19, 'total_tokens': 72})

In [16]:
prompt_with_history = RunnableWithMessageHistory(chain,get_session,input_messages_key="messages")
config={"configurable":{"session_id":"chat2"}}
prompt_with_history.invoke({
    "messages":[HumanMessage(content="Hi my name is krish naik")]},config=config)


AIMessage(content="Krish Naik, we're having a familiar conversation again. Is there something specific you'd like to talk about or ask?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 197, 'total_tokens': 224, 'completion_time': 0.043881683, 'completion_tokens_details': None, 'prompt_time': 0.012115976, 'prompt_tokens_details': None, 'queue_time': 0.048392214, 'total_time': 0.055997659}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--525be4a6-6b09-4e75-b59b-90e5e2b137b3-0', usage_metadata={'input_tokens': 197, 'output_tokens': 27, 'total_tokens': 224})

In [ ]:
prompt_with_history.invoke({
    "messages":[HumanMessage(content="what is my name")]},config=config)

AIMessage(content='Still your name, Krish Naik.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 274, 'total_tokens': 283, 'completion_time': 0.018369236, 'completion_tokens_details': None, 'prompt_time': 0.022668739, 'prompt_tokens_details': None, 'queue_time': 0.054310141, 'total_time': 0.041037975}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--5119cdba-bfcc-48fc-a6ad-151feff1b192-0', usage_metadata={'input_tokens': 274, 'output_tokens': 9, 'total_tokens': 283})

In [23]:
print(store["chat2"])

Human: Hi my name is krish naik
AI: Hello Krish Naik, it's nice to meet you. I'm here to help with any questions or information you need. How can I assist you today?
Human: Hi my name is krish naik
AI: Hello again Krish Naik. It seems I've already met you. How can I help you today? Do you have any specific questions or topics you'd like to discuss?
Human: what is my name
AI: Your name is Krish Naik.
Human: what is my name
AI: Your name is Krish Naik.
Human: Hi my name is krish naik
AI: Krish Naik, we're having a familiar conversation again. Is there something specific you'd like to talk about or ask?
Human: what is my name
AI: Your name is Krish Naik. We've had this conversation before, but I'm happy to confirm it again!
Human: what is my name
AI: Still your name, Krish Naik.
